In [1]:
# !pip install langchain chromadb  tiktoken pypdf langchain_groq langchain_community

In [2]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

In [3]:
doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )

In [4]:
docs = [doc1,doc2,doc3,doc4,doc5]

In [5]:
# Embedding Setup
embedding_model = HuggingFaceEmbeddings(
    model_name = 'intfloat/e5-small',
    encode_kwargs={'normalize_embeddings':True}
)

C:\Users\akash\AppData\Local\Temp\ipykernel_5884\2621033976.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
e:\15_CampusX_LangChain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Vector Store Setup
vector_store = Chroma(
    embedding_function = embedding_model,
    persist_directory='my_chroma_db',
    collection_name='sample'
)

C:\Users\akash\AppData\Local\Temp\ipykernel_5884\2905894603.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [7]:
# Add Documents -> we can pass our own id as well
vector_store.add_documents(docs)

['37904f34-7b3f-490a-9081-9a8796d91f38',
 'f60a5dab-457c-4063-bbf0-067728e5d076',
 'b6998d39-af1d-43f2-8b0e-9471e67826e7',
 '256a1ec1-dde5-4c95-84b7-c4e73e3b2c6a',
 '22a94aea-525b-4eb4-be8c-ab9722e6d85b']

In [8]:
# View documents
vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['37904f34-7b3f-490a-9081-9a8796d91f38',
  'f60a5dab-457c-4063-bbf0-067728e5d076',
  'b6998d39-af1d-43f2-8b0e-9471e67826e7',
  '256a1ec1-dde5-4c95-84b7-c4e73e3b2c6a',
  '22a94aea-525b-4eb4-be8c-ab9722e6d85b'],
 'embeddings': array([[ 0.01901846, -0.0149825 , -0.08518276, ...,  0.01487342,
         -0.00130043,  0.08019215],
        [ 0.01963638, -0.03145466, -0.07003246, ...,  0.00115768,
          0.00951148,  0.06269166],
        [ 0.01976115, -0.03853022, -0.12037989, ..., -0.01708872,
          0.02141127,  0.07051495],
        [-0.00270023, -0.02594806, -0.05797949, ..., -0.00857933,
          0.00691614,  0.02974596],
        [ 0.00416733, -0.04619418, -0.06100056, ...,  0.01147636,
          0.00969743,  0.0715995 ]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo

In [9]:
# Search Documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k = 2
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.')]

In [10]:
# Search with similarity Score
vector_store.similarity_search_with_score(
    query = "Who among these are bowler?",
    k=1
)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.34315598011016846)]

In [11]:
# Meta-data Filtering
vector_store.similarity_search_with_score(
    query="",
    filter = {'team':"Chennai Super Kings"}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.4943498373031616),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.5313642024993896)]

In [12]:
# Update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team":"Royal Challengers Bangalore"}
)
vector_store.update_document(document_id='f413bbc2-6aeb-4e09-87e1-b6ccaff004ed',document=updated_doc1)

In [13]:
vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['37904f34-7b3f-490a-9081-9a8796d91f38',
  'f60a5dab-457c-4063-bbf0-067728e5d076',
  'b6998d39-af1d-43f2-8b0e-9471e67826e7',
  '256a1ec1-dde5-4c95-84b7-c4e73e3b2c6a',
  '22a94aea-525b-4eb4-be8c-ab9722e6d85b'],
 'embeddings': array([[ 0.01901846, -0.0149825 , -0.08518276, ...,  0.01487342,
         -0.00130043,  0.08019215],
        [ 0.01963638, -0.03145466, -0.07003246, ...,  0.00115768,
          0.00951148,  0.06269166],
        [ 0.01976115, -0.03853022, -0.12037989, ..., -0.01708872,
          0.02141127,  0.07051495],
        [-0.00270023, -0.02594806, -0.05797949, ..., -0.00857933,
          0.00691614,  0.02974596],
        [ 0.00416733, -0.04619418, -0.06100056, ...,  0.01147636,
          0.00969743,  0.0715995 ]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo

In [14]:
# Delete Documents
vector_store.delete(ids=['dd63e793-9b42-40ad-aade-2ec03e4473ba'])

In [15]:
vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['37904f34-7b3f-490a-9081-9a8796d91f38',
  'f60a5dab-457c-4063-bbf0-067728e5d076',
  'b6998d39-af1d-43f2-8b0e-9471e67826e7',
  '256a1ec1-dde5-4c95-84b7-c4e73e3b2c6a',
  '22a94aea-525b-4eb4-be8c-ab9722e6d85b'],
 'embeddings': array([[ 0.01901846, -0.0149825 , -0.08518276, ...,  0.01487342,
         -0.00130043,  0.08019215],
        [ 0.01963638, -0.03145466, -0.07003246, ...,  0.00115768,
          0.00951148,  0.06269166],
        [ 0.01976115, -0.03853022, -0.12037989, ..., -0.01708872,
          0.02141127,  0.07051495],
        [-0.00270023, -0.02594806, -0.05797949, ..., -0.00857933,
          0.00691614,  0.02974596],
        [ 0.00416733, -0.04619418, -0.06100056, ...,  0.01147636,
          0.00969743,  0.0715995 ]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo